In [17]:
import numpy as np
import csv
import pandas as pd

from collections import Counter
from numpy import linalg as LA

In [18]:
with open("/content/drive/MyDrive/CIn/Mestrado/2021.1/Aprendizagem de Máquina/Projeto - Francisco/yeast.data") as input_file:
   lines = input_file.readlines()
   newLines = []
   for line in lines:
      newLine = line.strip().split()
      newLines.append(newLine)

with open("/content/drive/MyDrive/CIn/Mestrado/2021.1/Aprendizagem de Máquina/Projeto - Francisco/yeast.csv", 'w') as test_file:
   file_writer = csv.writer(test_file)
   file_writer.writerows(newLines)

In [19]:
# reading csv files
data =  pd.read_csv("/content/drive/MyDrive/CIn/Mestrado/2021.1/Aprendizagem de Máquina/Projeto - Francisco/yeast.csv",
                    names=["sequence_name", "mcg", "gvh", "alm", "mit", "erl", "pox", "vac", "nuc", "protein_local"])
print(data)

     sequence_name   mcg   gvh   alm   mit  erl  pox   vac   nuc protein_local
0       ADT1_YEAST  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22           MIT
1       ADT2_YEAST  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22           MIT
2       ADT3_YEAST  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22           MIT
3       AAR2_YEAST  0.58  0.44  0.57  0.13  0.5  0.0  0.54  0.22           NUC
4       AATM_YEAST  0.42  0.44  0.48  0.54  0.5  0.0  0.48  0.22           MIT
...            ...   ...   ...   ...   ...  ...  ...   ...   ...           ...
1479    YUR1_YEAST  0.81  0.62  0.43  0.17  0.5  0.0  0.53  0.22           ME2
1480    ZIP1_YEAST  0.47  0.43  0.61  0.40  0.5  0.0  0.48  0.47           NUC
1481    ZNRP_YEAST  0.67  0.57  0.36  0.19  0.5  0.0  0.56  0.22           ME2
1482    ZUO1_YEAST  0.43  0.40  0.60  0.16  0.5  0.0  0.53  0.39           NUC
1483    G6PD_YEAST  0.65  0.54  0.54  0.13  0.5  0.0  0.53  0.22           CYT

[1484 rows x 10 columns]


In [20]:
X = data.iloc[:, 1:9].values
y = data.iloc[:, 9].values

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score

inner_kf = KFold(n_splits=5)
outer_skf = StratifiedKFold(n_splits=5)
scaler = StandardScaler()
p_grid = {"knn__n_neighbors":range(2, 40, 1)}
knn = KNeighborsClassifier()
pipe = Pipeline(steps=[('scaler',scaler), ('knn', knn)])
clf=GridSearchCV(estimator=pipe, param_grid=p_grid, cv=inner_kf)
scoring = ['accuracy', 'f1_macro', 'recall_macro', 'precision_micro']
scores = cross_validate(clf, X=X, y=y, cv=outer_skf, scoring=scoring, return_estimator=True)

print("F1_macro; mean is: {}, deviation is:{}".format(scores['test_f1_macro'].mean(), scores['test_f1_macro'].std()))
print("Accuracy; mean is: {}, deviation is:{}".format(scores['test_accuracy'].mean(), scores['test_accuracy'].std()))
print("Precision_micro; mean is: {}, deviation is:{}".format(scores['test_precision_micro'].mean(), scores['test_precision_micro'].std()))
print("Recall_macro; mean is: {}, deviation is:{}".format(scores['test_recall_macro'].mean(), scores['test_recall_macro'].std()))

for estimator in scores['estimator']:
    print(estimator.best_params_)

F1_macro; mean is: 0.5163793726546485, deviation is:0.06349570529567317
Accuracy; mean is: 0.5727818727818728, deviation is:0.02575678124267269
Precision_micro; mean is: 0.5727818727818728, deviation is:0.02575678124267269
Recall_macro; mean is: 0.5242628683664616, deviation is:0.07177279178446955
{'knn__n_neighbors': 9}
{'knn__n_neighbors': 12}
{'knn__n_neighbors': 14}
{'knn__n_neighbors': 18}
{'knn__n_neighbors': 14}


### Escolhendo o melhor hiperparametro

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

skf = StratifiedKFold(n_splits=5)

k_list = [9,12,14,18]

mean_accuracy = []

for k in k_list:

  accuracy = []
  precision = []
  recall = []
  f_measure = []

  for train_index, test_index in skf.split(X, y):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, y_train)

    knn_predict = neigh.predict(X_test)

    accuracy.append(accuracy_score(y_test, knn_predict))
    precision.append(precision_score(y_test, knn_predict, average='macro',zero_division=0))
    recall.append(recall_score(y_test, knn_predict, average='macro',zero_division=0))
    f_measure.append(f1_score(y_test, knn_predict, average='macro',zero_division=0))

  print('\n')
  print(k)
  print("Mean accuracy: ",sum(accuracy)/len(accuracy), "Std: ",np.std(accuracy))
  print("Mean precision: ",sum(precision)/len(precision), "Std: ",np.std(precision))
  print("Mean recall: ",sum(recall)/len(recall), "Std: ",np.std(recall))
  print("Mean f_measure: ",sum(f_measure)/len(f_measure), "Std: ",np.std(f_measure))

  mean_accuracy.append([sum(accuracy)/len(accuracy),k])

print("\nMax accuracy: ",max(mean_accuracy)[0],"Parameter: ",max(mean_accuracy)[1])



9
Mean accuracy:  0.5674196924196924 Std:  0.03358772709312427
Mean precision:  0.5802622293124762 Std:  0.055241357826057706
Mean recall:  0.5691200620534274 Std:  0.03693778283303979
Mean f_measure:  0.554191222376502 Std:  0.04362157311294963


12
Mean accuracy:  0.5613386113386113 Std:  0.02391171939133774
Mean precision:  0.5518903557812654 Std:  0.03851880571454237
Mean recall:  0.5260904046865156 Std:  0.042387396701307554
Mean f_measure:  0.524174629080318 Std:  0.03633212941554262


14
Mean accuracy:  0.5687369187369187 Std:  0.02430044763606055
Mean precision:  0.5705006564236044 Std:  0.043049057247285784
Mean recall:  0.5289184569337679 Std:  0.050444272756627856
Mean f_measure:  0.5342244172609023 Std:  0.04542437717118001


18
Mean accuracy:  0.5802006552006553 Std:  0.015851871725002156
Mean precision:  0.506433319908586 Std:  0.031143095940247514
Mean recall:  0.46418380316835306 Std:  0.023423849708725027
Mean f_measure:  0.46871241656436285 Std:  0.02346386082755748